# Data preparation with Pandas

Most of the datasets we are dealing with can be considered "semi-structured", i.e., even if they do not conform to a data model, they have an identifiable structure and they have some organisational properties that make it easier to analyze. Nevertheless, they can be still considered as **raw data**, since they may contain errors, missing values, or partial information. 

Examples of these datasets are:
- logs produced by a web server, containing the user activity (e.g., clicks);
- records produced by a network traffic analyzer, that monitors the network activities and produces summaries (e.g., packet exchange);
- emails that are analyzed by a intrusion detection system to understand if there is an attack.

Semi-structured data can be organized into a set of information with labels or tags, or in a tabular form (which can be the output of a pre-processing). 

In this notebook we will analyze a semi-structured dataset in a tabular form with the help of **Pandas**, which is a Python library for data manipulation and analysis. Students not familiar with Pandas may find some introductory tutorials here:

https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html  
https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html  
https://github.com/jvns/pandas-cookbook

In the following, we first introduce our dataset, and then we show some common exploratory data analysis that is done on raw datasets.


## The dataset - Airline data

We consider the data related to airline activities that can be found at  
http://stat-computing.org/dataexpo/2009/the-data.html

It contains 29 fields, that can be either categorical or numerical. For example, the ```Origin``` (source airport) is categorical. The ```DepTime``` (departure time) feature is numerical. For instance, "flight departing before 6PM" can be express by "DepTime < 1800". The structure of each line is as follows:

```
1	 Year	1987-2008
2	 Month	1-12
3	 DayofMonth	1-31
4	 DayOfWeek	1 (Monday) - 7 (Sunday)
5	 DepTime	actual departure time (local, hhmm)
6	 CRSDepTime	scheduled departure time (local, hhmm)
7	 ArrTime	actual arrival time (local, hhmm)
8	 CRSArrTime	scheduled arrival time (local, hhmm)
9	 UniqueCarrier	unique carrier code
10	 FlightNum	flight number
11	 TailNum	plane tail number
12	 ActualElapsedTime	in minutes
13	 CRSElapsedTime	in minutes
14	 AirTime	in minutes
15	 ArrDelay	arrival delay, in minutes
16	 DepDelay	departure delay, in minutes
17	 Origin	origin IATA airport code
18	 Dest	destination IATA airport code
19	 Distance	in miles
20	 TaxiIn	taxi in time, in minutes
21	 TaxiOut	taxi out time in minutes
22	 Cancelled	was the flight cancelled?
23	 CancellationCode	reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
24	 Diverted	1 = yes, 0 = no
25	 CarrierDelay	in minutes
26	 WeatherDelay	in minutes
27	 NASDelay	in minutes
28	 SecurityDelay	in minutes
29	 LateAircraftDelay	in minutes
```

There is a single CSV file per year. Each file contains a header that serves the purpose of an "embedded schema", to help data scientist figure out what information is available. This is what the beginning of a file looks like:

```
Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1994,1,7,5,858,900,954,1003,US,227,NA,56,63,NA,-9,-2,CLT,ORF,290,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1994,1,8,6,859,900,952,1003,US,227,NA,53,63,NA,-11,-1,CLT,ORF,290,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1994,1,10,1,935,900,1023,1003,US,227,NA,48,63,NA,20,35,CLT,ORF,290,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1994,1,11,2,903,900,1131,1003,US,227,NA,148,63,NA,88,3,CLT,ORF,290,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1994,1,12,3,933,900,1024,1003,US,227,NA,51,63,NA,21,33,CLT,ORF,290,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1994,1,13,4,NA,900,NA,1003,US,227,NA,NA,63,NA,NA,NA,CLT,ORF,290,NA,NA,1,NA,0,NA,NA,NA,NA,NA
```

Note that there are some fields with missing values in some lines of the dataset. The missing values are marked by "NA". These values can cause problems when processing the data and can lead to unexpected results. 
 
When analyzing the dataset, there are many intial questions that could be of insterest,  such as:

 - How many unique origin airports?
 - How many unique destination airports?
 - How many carriers?
 - How many night flights do we have in our data? ("night" starts at 6pm)
 - How many night flights per unique carrier?
 - ...
 
In order to answer to such questions, we need to check our dataset and remove the causes of potential problems. 

## Data loading and initial exploration

To understand if the dataset needs to be processed, a convenient approach is to consider a **random sample**. This will provide an indication about the quality of the dataset and, if there are some operations to do (e.g., remove lines with some missing values), the sample will provide a way to understand the *pattern* to follow.

There are different ways for producing a sample, in our case we will work on a single year - an alternative approach would be to sample a set of lines from the different years.

The dataset is organized as csv file, and Pandas has a method to load such a type of file (it considers the first row as the header that contains the name of the fields).

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

# We read directly the compressed file
input_file = "7-1994.csv.gz"
dataset = pd.read_csv(input_file, compression='gzip')

# Check how many rows and columsn the dataset contains
num_rows, num_cols = dataset.shape
print("The dataset contains", num_rows, "rows and", num_cols, "columns")

We can look at the first 10 rows (for ease of reading, we show the transpose, so that the columns becomes rows) 

In [ ]:
dataset.head(10).T

We can obtain additional information about each column with the following command:

In [ ]:
dataset.info()

Pandas compute some basic statistics on the values in each column with ```.describe()``` - also in this case, we show the transpose for ease of reading.

In [ ]:
dataset.describe().T

## Analysis of the columns

We start looking for columns that contains missing values

In [ ]:
columns_with_miss = dataset.isna().sum()

# Filtering only the columns with at least 1 missing value
columns_with_miss = columns_with_miss[columns_with_miss!=0]
print('Columns with missing values:', len(columns_with_miss))

# Show the columns with missing values, sorted in descending order
columns_with_miss.sort_values(ascending=False)

Clearly there are 10 columns that do not contain any value (recall, the dataset has 5180048 rows), so we can remove them:

In [ ]:
dataset.drop(columns=['LateAircraftDelay', 'SecurityDelay', \
 'NASDelay', 'WeatherDelay', 'CarrierDelay', 'CancellationCode', \
 'TaxiOut', 'TaxiIn', 'AirTime', 'TailNum'], inplace=True)

We will take care of the other coulmns with missing values (```ArrDelay```, ```ActualElapsedTime```, ...) when we will analyze the rows.

Besides missing values, we have also values equal to zero, so we need to check them and see if their values make sense:

In [ ]:
dataset.isin([0]).sum().sort_values(ascending=False)

Most of them makes sense, except for ```ActualElapsedTime```, since it is stange that the flight time is equal to zero. We will consider it when analyzing the rows.

We conclude this column-wise exploration by looking (visually) for outliers.

In [ ]:
numerical_col = \
list(dataset.dtypes[dataset.dtypes == 'int64'].index) + \
list(dataset.dtypes[dataset.dtypes == 'float64'].index)

len(numerical_col)

In [ ]:
import seaborn as sns

rows = 4
columns = 4

fig, axes = plt.subplots(rows,columns, figsize=(30,30))

x, y = 0, 0

for i, column in enumerate(numerical_col):
    sns.boxplot(x=dataset[column], ax=axes[x, y])
    
    if y < columns-1:
        y += 1
    elif y == columns-1:
        x += 1
        y = 0
    else:
        y += 1

There may be some outlier in ```Distance```, in which case we should analyze the specific rows and, in case, remove them (we leave this task to the student).

## Analysis of the rows

We now inspect the rows that we identified as suspicious during the general analysis of the columns, i.e., rows that contains NA or 0.

For instance, we have seen that there are 11 rows in which the field ```ActualElapsedTime``` is zero. 

In [ ]:
dataset[dataset['ActualElapsedTime'].isin([0])].T

The problem is clearly due to an error in recording the departure and arrival times. We could adjust the values, since we have the scheduled departure and scheduled arrival times, along with the departure and arrival delays. But since there are only 11 records, it's faster to drop them.

In [ ]:
index_to_drop =  dataset[dataset['ActualElapsedTime'].isin([0])].index
dataset = dataset.drop(index_to_drop)

In the same way, we can inspect the rows with NA in the fields 

|Field |Count|
|:-----|----:|
|ArrDelay               |78846|
|ActualElapsedTime      |78846|
|ArrTime                |78846|
|DepDelay               |66740|
|DepTime                |66740|
|Distance               |22949|

Notice how some counts are the same.

In [ ]:
dataset[dataset['ActualElapsedTime'].isna()].head(10).T

For security, we remove the rows in which both ```ActualElapsedTime``` and ```ArrTime``` is NA, since without these two values we could not recover the data.

In [ ]:
index_to_drop =  dataset[dataset['ArrTime'].isna() & dataset['ActualElapsedTime'].isna()].index
dataset = dataset.drop(index_to_drop)

We now check if there are still NA values.

In [ ]:
print("Records with NA in ActualElapsedTime:", dataset['ActualElapsedTime'].isna().sum())
print("Records with NA in ArrDelay:", dataset['ArrDelay'].isna().sum())
print("Records with NA in ArrTime:", dataset['ArrTime'].isna().sum())
print("Records with NA in DepDelay:", dataset['DepDelay'].isna().sum())
print("Records with NA in DepTime:", dataset['DepTime'].isna().sum())
print("Records with NA in Distance:", dataset['Distance'].isna().sum())

As exercise, check if the records with ```Distance``` equal to NA should be dropped.

## Data normalization

Once the data has been cleaned (including the removal of outliers), we can look at the distribution of the numerical values and see if we need to normalize them. **In this specific dataset, normalization is not required**, since fields refer to times and distances. But we will normalize the values of a column as representative example. 

In [ ]:
# Plot the empirical distribution of the field "ActualElapsedTime"
dataset['ActualElapsedTime'].plot.hist(bins=100) 

We can normalize with respect to the maximum value (in this case, we make a copy of the column beacuse we do no want to modify the dataset):

In [ ]:
AET_copy = dataset['ActualElapsedTime'].copy()
AET_copy = AET_copy/AET_copy.max()
AET_copy.plot.hist(bins=100) 

Alternatively, we can normalize with respect to the minimum and maximum values:

In [ ]:
AET_copy = dataset['ActualElapsedTime'].copy()
AET_copy = (AET_copy - AET_copy.min())/(AET_copy.max() - AET_copy.min())
AET_copy.plot.hist(bins=100) 

Another option is to compute the z-score

In [ ]:
AET_copy = dataset['ActualElapsedTime'].copy()
AET_copy = (AET_copy - AET_copy.mean())/AET_copy.std()
AET_copy.plot.hist(bins=100) 

## Final considerations

Once the dataset has been processed, we can save it with the command:
```
dataset.to_csv('1994-cleaned.csv')
```

Clearly, there are other pre-processing steps that can be done (e.g., check that the flight time is indeed the difference between the arrival and departure time), but such steps are specific to the dataset to be analyzed. In this short notebook, we have simply shown come general approach to data preparation, with common steps usualy adopted in such a process.